In [ ]:
import matching
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm
import sys
sys.path.append('/home/yuanlq/xenon/compeaks')
import comparison

In [ ]:
import straxen
straxen.print_versions()

In [ ]:
COMPARISON_SPACES1D = ['z', 's1_area', 's1_area_fraction_top', 's1_range_50p_area', 's1_range_90p_area',
                       's1_rise_time', 's1_n_channels', 's1_tight_coincidence', 's1_n_hits']
COMPARISON_SPACES2D = [('z', 's1_area_fraction_top'),
                       ('z', 's1_rise_time'), 
                       ('z', 's1_range_50p_area'),
                       ('z', 's1_range_90p_area'),
                       ('z', 's1_area'),
                       ('z', 's1_tight_coincidence'),
                       ('s1_area_fraction_top','s1_rise_time'),
                       ('s1_area', 's1_range_50p_area'),
                       ('s1_area', 's1_rise_time')]

# Data Loading

In [ ]:
ambe_sr0 = np.load("/project2/lgrandi/yuanlq/shared/sr0_ambe_ss.npy")
ambe_sr0 = ambe_sr0[ambe_sr0['z']>-150]
ambe_sr0 = ambe_sr0[ambe_sr0['z']<-0.1]
ambe_sr0 = ambe_sr0[ambe_sr0['s1_area']<=50]
ambe_sr0 = pd.DataFrame(ambe_sr0)
""" Need to add SS cuts
daniel1 = np.load('/project2/lgrandi/wenz/strax_data/sr1/ambe/events_topCW5d9m_nv_coincidence_applied.npy')
daniel2 = np.load('/project2/lgrandi/wenz/strax_data/sr1/ambe/events_topCW7d8m_nv_coincidence_applied.npy')
ambe_sr1 = np.concatenate((daniel1, daniel2))
ambe_sr1 = ambe_sr1[ambe_sr1['z']>-150]
ambe_sr1 = ambe_sr1[ambe_sr1['z']<-0.1]
ambe_sr1 = ambe_sr1[ambe_sr1['s1_area']<50]
ambe_sr1 = pd.DataFrame(ambe_sr1)
"""

In [ ]:
# Get all SR0 Ar37
"""
import cutax
st = cutax.xenonnt_v8(output_folder="/project2/lgrandi/xenonnt/processed")
ar_runs = st.select_runs(run_mode='tpc_ar37', available="event_info").name
st.register_all(cutax.cut_lists.ar37)

available=[]
for run in ar_runs:
    if st.is_stored(run, 'event_info'):
        available.append(True)
    else:
        available.append(False)
available=np.array(available)
ar_runs = ar_runs[available]

for i,run in tqdm(enumerate(ar_runs)):
    if i==0:
        ar37 = st.get_array(run, ('event_info', 'cuts_ar37_kshell_s1s2'))
        ar37 = ar37[ar37['cuts_ar37_kshell_s1s2']]
    else:
        new = st.get_array(run, ('event_info', 'cuts_ar37_kshell_s1s2'))
        new = new[new['cuts_ar37_kshell_s1s2']]
        ar37 = np.concatenate((ar37, new))
    gc.collect()

np.save('/project2/lgrandi/yuanlq/shared/sr0_ar37_k.npy',ar37)
"""
ar37_sr0 = np.load('/project2/lgrandi/yuanlq/shared/sr0_ar37_k.npy')
ar37_sr0 = ar37_sr0[ar37_sr0['z']>-150]
ar37_sr0 = ar37_sr0[ar37_sr0['z']<-0.1]
ar37_sr0 = pd.DataFrame(ar37_sr0)

In [ ]:
len(ar37_sr0)

In [ ]:
len(ambe_sr0)

# Before Matching

In [ ]:
comparison.compare_2para(peak_extra0=ar37_sr0, 
                         peak_extra1=ambe_sr0, 
                         signal_type0='Ar37 SR0', 
                         signal_type1='SS AmBe SR0', 
                         comparison_spaces = COMPARISON_SPACES2D,
                         errorbar = 'mean_error')

In [ ]:
comparison.compare_1para(peak_extra0=ar37_sr0,
                         peak_extra1=ambe_sr0,
                         signal_type0='Ar37 SR0',
                         signal_type1='SS AmBe SR0',
                         comparison_spaces = COMPARISON_SPACES1D,
                         n_x = 50
                         )

# After Matching

Here we treat Ar37 as "simulation" and SS AmBe as "data".

Matching: `s1_area_fraction_top`, `s1_n_hits`, `z`

## Match AmBe to Ar37 

In [ ]:
inf = matching.inference.Inference(data=ambe_sr0, simu=ar37_sr0[:300000], 
                                   covariates=['s1_area_fraction_top', 's1_n_hits', 'z'])

In [ ]:
matched_ambe = inf.match_simu()

In [ ]:
comparison.compare_2para(peak_extra0=ar37_sr0[:300000], 
                         peak_extra1=matched_ambe, 
                         signal_type0='Ar37 SR0', 
                         signal_type1='Matched SS AmBe SR0', 
                         comparison_spaces = COMPARISON_SPACES2D,
                         errorbar = 'mean_error')

In [ ]:
comparison.compare_1para(peak_extra0=ar37_sr0[:300000],
                         peak_extra1=matched_ambe,
                         signal_type0='Ar37 SR0',
                         signal_type1='Matched SS AmBe SR0',
                         comparison_spaces = COMPARISON_SPACES1D,
                         n_x = 50
                         )

## Match Ar37 to AmBe

In [ ]:
inf = matching.inference.Inference(data=ar37_sr0[:300000], simu=ambe_sr0, 
                                   covariates=['s1_area_fraction_top', 's1_n_hits', 'z'])

In [ ]:
matched_ar37 = inf.match_simu()

In [ ]:
comparison.compare_2para(peak_extra0= matched_ar37, 
                         peak_extra1= ambe_sr0, 
                         signal_type0='Matched Ar37 SR0', 
                         signal_type1='SS AmBe SR0', 
                         comparison_spaces = COMPARISON_SPACES2D,
                         errorbar = 'mean_error')

In [ ]:
comparison.compare_1para(peak_extra0= matched_ar37, 
                         peak_extra1= ambe_sr0, 
                         signal_type0='Matched Ar37 SR0', 
                         signal_type1='SS AmBe SR0', 
                         comparison_spaces = COMPARISON_SPACES1D,
                         n_x = 50
                         )